## **Przykłady: Multi-AI Agenci do rezerwacji hotelu**

W dzisiejszym szybkim tempie życia planowanie podróży służbowej to coś więcej niż tylko rezerwacja lotu i pokoju hotelowego. Wymaga to poziomu koordynacji i efektywności, który może być trudny do osiągnięcia. Właśnie tutaj wkraczają Multi-AI Agenci, rewolucjonizując sposób, w jaki zarządzamy naszymi potrzebami podróżnymi.

Wyobraź sobie zespół inteligentnych agentów do Twojej dyspozycji, współpracujących, aby zająć się każdym aspektem Twojej podróży z precyzją i łatwością. Dzięki naszej zaawansowanej technologii AI stworzyliśmy wyspecjalizowanych agentów do rezerwacji usług i organizacji planu podróży, zapewniając płynne i bezstresowe doświadczenie podróżne.

To jest podstawowy scenariusz. Planując podróż służbową, musimy skonsultować się z agentem podróży służbowych, aby uzyskać informacje o biletach lotniczych, hotelach itp. Dzięki AI Agentom możemy stworzyć agentów do rezerwacji usług oraz agentów do organizacji planu podróży, którzy będą współpracować i podnosić poziom inteligencji.


# Inicjalizacja usługi Azure AI Agent i pobranie informacji konfiguracyjnych z **.env**

### **.env** 

Utwórz plik .env 

Plik **.env** zawiera ciąg połączenia do usługi Azure AI Agent, model używany przez AOAI oraz odpowiednie API wyszukiwania Google, ENDPOINT itp.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Nazwa wdrożenia modelu usługi Azure AI Agent"

[**NOTE**] Będziesz potrzebować modelu z limitem 100 000 tokenów na minutę oraz limitem 600 żądań na minutę.

  Model można uzyskać w Azure AI Foundry - Model and Endpoint. 

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Ciąg połączenia projektu usługi Azure AI Agent"

  Ciąg połączenia projektu można znaleźć w przeglądzie projektu na ekranie AI Foundry Portal.

- **SERPAPI_SEARCH_API_KEY** = "Twój klucz API wyszukiwania SERPAPI"
- **SERPAPI_SEARCH_ENDPOINT** = "Twój endpoint wyszukiwania SERPAPI"

Aby uzyskać nazwę wdrożenia modelu i ciąg połączenia projektu usługi Azure AI Agent, musisz utworzyć usługę Azure AI Agent. Zaleca się użycie [tego szablonu](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) do bezpośredniego utworzenia usługi (***Uwaga:*** Usługa Azure AI Agent jest obecnie dostępna w ograniczonych regionach. Zaleca się zapoznanie z [tym linkiem](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) w celu ustawienia regionu).

Agent musi mieć dostęp do SERPAPI. Zaleca się rejestrację za pomocą [tego linku](https://serpapi.com/searches). Po rejestracji możesz uzyskać unikalny klucz API i ENDPOINT.


# Logowanie do Azure

Teraz musisz zalogować się do Azure. Otwórz terminal w VScode i uruchom polecenie `az login`.


# Konfiguracja

Aby uruchomić ten notebook, musisz zainstalować następujące biblioteki. Oto lista wymaganych bibliotek oraz odpowiadające im polecenia instalacji za pomocą pip:

azure-identity: Do uwierzytelniania w Azure.  
requests: Do wykonywania żądań HTTP.  
semantic-kernel: Do frameworka semantic kernel (zakładając, że jest to niestandardowa lub specyficzna biblioteka, może być konieczne zainstalowanie jej z określonego źródła lub repozytorium).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Wyjaśnienie: 
import asyncio: Importuje moduł asyncio, który wspiera programowanie asynchroniczne w Pythonie. Umożliwia pisanie kodu współbieżnego przy użyciu składni async i await.
from typing: Annotated: Importuje typ Annotated z modułu typing. Annotated służy do dodawania metadanych do wskazówek typów, co może być przydatne w różnych celach, takich jak walidacja, dokumentacja czy narzędzia.


In [ ]:
import asyncio,os
from typing import Annotated

# Wyjaśnienie:
Korzystając z `from dotenv import load_dotenv` oraz `load_dotenv()`, możesz w prosty sposób zarządzać ustawieniami konfiguracyjnymi i poufnymi informacjami (takimi jak klucze API czy adresy URL baz danych) w pliku `.env`, oddzielając je od kodu źródłowego. Dzięki temu Twoja aplikacja staje się bardziej bezpieczna i łatwiejsza w konfiguracji.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Wyjaśnienie:

Importowanie: from azure.identity.aio import DefaultAzureCredential: Ten import wprowadza klasę DefaultAzureCredential z modułu azure.identity.aio. Część "aio" w nazwie modułu wskazuje, że jest on przeznaczony do operacji asynchronicznych.

Cel DefaultAzureCredential: Klasa DefaultAzureCredential jest częścią Azure SDK dla Pythona. Zapewnia domyślny sposób uwierzytelniania w usługach Azure. Próbuje uwierzytelnić się za pomocą wielu metod w określonej kolejności, takich jak zmienne środowiskowe, zarządzana tożsamość i poświadczenia Azure CLI.

Operacje asynchroniczne: Moduł "aio" wskazuje, że klasa DefaultAzureCredential obsługuje operacje asynchroniczne. Oznacza to, że można ją używać z asyncio do wykonywania nieblokujących żądań uwierzytelniania.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Wyjaśnienie:
Importuje różne moduły i klasy z pakietu semantic_kernel. Oto szczegółowy opis każdego importu:

AgentGroupChat z semantic_kernel.agents: Ta klasa obsługuje funkcje związane z czatem grupowym dla agentów AI. AzureAIAgent i AzureAIAgentSettings z semantic_kernel.agents.azure_ai

AzureAIAgent: Ta klasa służy do tworzenia i zarządzania agentami AI korzystającymi z usług Azure AI.

AzureAIAgentSettings: Ta klasa służy do konfigurowania ustawień dla AzureAIAgent. TerminationStrategy z semantic_kernel.agents.strategies.termination.termination_strategy:

Ta klasa definiuje strategie zakończenia działania agentów AI w określonych warunkach. ChatMessageContent z semantic_kernel.contents.chat_message_content:

Ta klasa służy do obsługi treści wiadomości czatu.
AuthorRole z semantic_kernel.contents.utils.author_role:

Ta klasa definiuje różne role autorów w kontekście wiadomości czatu. 

kernel_function z semantic_kernel.functions.kernel_function_decorator: Ten dekorator służy do definiowania funkcji jądra, czyli funkcji, które mogą być wykonywane w ramach frameworka semantic kernel.
Te importy przygotowują niezbędne komponenty do tworzenia i zarządzania agentami AI, którzy mogą wchodzić w interakcje w środowisku czatu grupowego, na przykład w celu realizacji zadań takich jak rezerwacja hoteli lub podobne działania.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Wyjaśnienie:
Następnie importujemy klasę CodeInterpreterTool z modułu azure.ai.projects.models.

CodeInterpreterTool: Ta klasa jest częścią Azure AI SDK i służy do interpretacji oraz wykonywania kodu w kontekście projektów AI. Udostępnia funkcje umożliwiające uruchamianie fragmentów kodu, analizowanie kodu lub integrowanie wykonywania kodu w ramach przepływów pracy związanych ze sztuczną inteligencją.  
Ten import przygotowuje niezbędny komponent do korzystania z CodeInterpreterTool w Twoim projekcie, co może być przydatne w zadaniach wymagających dynamicznej interpretacji i wykonywania kodu.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Wyjaśnienie: 
Klasa ApprovalTerminationStrategy dostarcza konkretną strategię zakończenia działania agenta AI. Agent zakończy swoje działanie, jeśli ostatnia wiadomość w jego historii interakcji zawiera słowo „zapisano”. Może to być przydatne w sytuacjach, gdy zadanie agenta uznaje się za zakończone po otrzymaniu potwierdzenia, że coś zostało „zapisane”. Zdefiniuj metodę interakcji. Po zapisaniu planu rezerwacji można zatrzymać działanie agenta po otrzymaniu sygnału „zapisano”.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Wyjaśnienie:

Linia kodu inicjalizuje obiekt AzureAIAgentSettings z domyślnymi lub predefiniowanymi ustawieniami, wywołując metodę create(). Ten obiekt ustawień (ai_agent_settings) może być następnie użyty do konfiguracji i zarządzania instancją AzureAIAgent.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Wyjaśnienie:
Dzięki zaimportowaniu biblioteki requests możesz łatwo wykonywać żądania HTTP i korzystać z usług internetowych w swoim kodzie Python.


In [ ]:
import requests

# Wyjaśnienie:
To zmienna, która przechowuje klucz API umożliwiający dostęp do usługi API SERP (Search Engine Results Page). Klucz API to unikalny identyfikator używany do uwierzytelniania żądań powiązanych z Twoim kontem.

'GOOGLE_SEARCH_API_KEY': To jest ciąg zastępczy. Musisz zastąpić 'GOOGLE_SEARCH_API_KEY' swoim rzeczywistym kluczem API SERP.

Cel: Celem tej linii jest przechowywanie klucza API w zmiennej, aby można go było używać do uwierzytelniania żądań do usługi API SERP. Klucz API jest wymagany, aby uzyskać dostęp do usługi i wykonywać wyszukiwania.

Jak uzyskać klucz API SERP: Aby uzyskać klucz API SERP, wykonaj następujące ogólne kroki na stronie https://serpapi.com (dokładne kroki mogą się różnić w zależności od konkretnej usługi API SERP, z której korzystasz):

Wybierz usługę API SERP: Istnieje kilka dostępnych usług API SERP, takich jak SerpAPI, Google Custom Search JSON API i inne. Wybierz tę, która najlepiej odpowiada Twoim potrzebom.

Zarejestruj konto:

Przejdź na stronę wybranej usługi API SERP https://www.serpapi.com i zarejestruj konto. Może być konieczne podanie podstawowych informacji i zweryfikowanie adresu e-mail.

Utwórz klucz API:

Po rejestracji zaloguj się na swoje konto i przejdź do sekcji API lub pulpitu nawigacyjnego. Poszukaj opcji utworzenia lub wygenerowania nowego klucza API.
Skopiuj klucz API:

Gdy klucz API zostanie wygenerowany, skopiuj go. Ten klucz będzie używany do uwierzytelniania Twoich żądań do usługi API SERP.
Zastąp ciąg zastępczy:

Zastąp ciąg zastępczy w swoim pliku .env


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Wyjaśnienie:
Klasa BookingPlugin udostępnia metody do rezerwacji hoteli i lotów za pomocą Google Search API od Serpapi.com. Tworzy niezbędne parametry, wysyła żądania do API i przetwarza odpowiedzi, aby zwrócić odpowiednie informacje o rezerwacjach. Klucz API (SERPAPI_SEARCH_API_KEY) oraz punkt końcowy (SERPAPI_SEARCH_ENDPOINT) są wykorzystywane do uwierzytelniania i wysyłania żądań do Google Search API.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Wyjaśnienie:
Klasa SavePlugin udostępnia metodę saving_plan do zapisywania planów podróży za pomocą usług Azure AI. Ustawia dane uwierzytelniające Azure, tworzy agenta AI, przetwarza dane wejściowe użytkownika w celu wygenerowania i zapisania treści planu podróży, a także obsługuje operacje zapisywania plików i czyszczenia. Metoda zwraca "Saved" po pomyślnym zakończeniu.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Wyjaśnienie:
Ten kod konfiguruje agentów Azure AI do obsługi rezerwacji lotów i hoteli oraz zapisywania planów podróży na podstawie danych wprowadzonych przez użytkownika. Wykorzystuje poświadczenia Azure do tworzenia i konfiguracji agentów, przetwarza dane użytkownika za pomocą czatu grupowego i zapewnia odpowiednie czyszczenie po zakończeniu zadań. Agenci korzystają z określonych wtyczek (BookingPlugin i SavePlugin) do wykonywania swoich zadań.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Zastrzeżenie**:  
Ten dokument został przetłumaczony za pomocą usługi tłumaczenia AI [Co-op Translator](https://github.com/Azure/co-op-translator). Chociaż dokładamy wszelkich starań, aby tłumaczenie było precyzyjne, prosimy pamiętać, że automatyczne tłumaczenia mogą zawierać błędy lub nieścisłości. Oryginalny dokument w jego rodzimym języku powinien być uznawany za wiarygodne źródło. W przypadku informacji o kluczowym znaczeniu zaleca się skorzystanie z profesjonalnego tłumaczenia przez człowieka. Nie ponosimy odpowiedzialności za jakiekolwiek nieporozumienia lub błędne interpretacje wynikające z użycia tego tłumaczenia.
